In [ ]:
import torch
import torch.nn as nn

import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

from Utils import MultilayerPerceptron

**Reading the Dataset**

In [ ]:
dataset = pd.read_csv('./data.csv')
dataset_label = pd.read_csv('./label.csv')

**Normalizing the data wiith MinMax scaler**

In [ ]:
scaler = MinMaxScaler()
scaler.fit(dataset)
dataset = scaler.transform(dataset)

**Converting the dataset and labels to Torch Tensor and creating the TensorDataset**

In [ ]:
train_dataset = TensorDataset(torch.FloatTensor(dataset), torch.LongTensor(dataset_label.values))

**Initializing the model, seeting the criterion, optimizer, and learniong rate**

In [ ]:
torch.manual_seed(110)
model = MultilayerPerceptron()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

**Do the training with the desired number of epochs and save the model with the highest accuracy**

In [ ]:
trainloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
epochs = 10
acc = 0
for i in range(epochs):
    i+=1
    print(f'Training epoch {i}:')
    
    for b, (X_train, y_train) in enumerate(trainloader):
        b+=1
        if b % 5000 == 0:
            print(f'Batch number {b}')
        
        y_pred = model(X_train)
        loss = criterion(y_pred, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    y_pred = model(torch.FloatTensor(dataset))
    true = (torch.LongTensor(dataset_label.values) == torch.max(y_pred.data, 1)[1]).sum()
    
    print(f'epoch: {i:2}  loss: {loss.item():10.8f}  accuracy: {(true.item()/dataset_label.shape[0])*100:6.4f}')
    if(true.item()/dataset_label.shape[0] > acc):
        torch.save(model.state_dict(), "DNN.pt")
        acc = true.item()/dataset_label.shape[0]
        print(f'Model saved with accuracy: {acc*100:6.4f}')